<a href="https://colab.research.google.com/github/abhranil-datascience/ClassificationPOC/blob/master/DecisionTreeClassifierPOC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
############################## Mount Drive ######################################## 
from google.colab import drive
drive.mount('/content/gdrive')
############################## Change Directory ###################################
import os
os.chdir('/content/gdrive/My Drive/MLandDLFullCourse/ML/Classification/DecisionTreeClassifier')
########## Suppress Warnings ############
import warnings
warnings.filterwarnings('ignore')
######### Import Dataset ###############
import pandas as pd
dataset=pd.read_csv('Social_Network_Ads.csv')
X=dataset.iloc[:,1:4].values
Y=dataset.iloc[:,4].values
######## Categorical Variable ############
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
X[:,0]=le.fit_transform(X[:,0])
######## Scaling ############
from sklearn.preprocessing import MinMaxScaler
sc=MinMaxScaler(feature_range=(0,1))
XScaled=sc.fit_transform(X)
######## Test Train Split #########
from sklearn.model_selection import train_test_split
XTrain,XTest,YTrain,YTest=train_test_split(XScaled,Y,test_size=0.1,random_state=42)
######## Check VIF ###############
VIF=[0]*XTrain.shape[1]
from statsmodels.stats.outliers_influence import variance_inflation_factor
for col in range(0,XTrain.shape[1]):
  VIF[col]=VIF[col]+variance_inflation_factor(exog=XTrain,exog_idx=col)#1.6142518559270171, 2.9082305467018816, 2.753146445004674
VIF

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


[1.6142518559270171, 2.9082305467018816, 2.753146445004674]

In [0]:
######## Grid Search For Hyperparameter Tuning ############
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
RegressorDT_GS=DecisionTreeClassifier()
params_GS=[{'min_impurity_split':[1e-8,5e-8,1e-7,5e-7,1e-6],
           'criterion':['gini','entropy'],
           'splitter':['best','random']}]
GS=GridSearchCV(estimator=RegressorDT_GS,param_grid=params_GS,scoring='accuracy',cv=10)
GS.fit(XTrain,YTrain)
best_params_GS=GS.best_params_#'criterion': 'gini', 'min_impurity_split': 1e-06, 'splitter': 'random'
best_params_GS

/usr/local/lib/python3.6/dist-packages/sklearn/tree/tree.py:297: DeprecationWarning: The min_impurity_split parameter is deprecated. Its default value will change from 1e-7 to 0 in version 0.23, and it will be removed in 0.25. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/tree/tree.py:297: DeprecationWarning: The min_impurity_split parameter is deprecated. Its default value will change from 1e-7 to 0 in version 0.23, and it will be removed in 0.25. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/tree/tree.py:297: DeprecationWarning: The min_impurity_split parameter is deprecated. Its default value will change from 1e-7 to 0 in version 0.23, and it will be removed in 0.25. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/tree/tree.py:297: DeprecationWarning: The min_impurity_spli

{'criterion': 'gini', 'min_impurity_split': 1e-07, 'splitter': 'random'}

In [0]:
######## ApplyKFold #############
RegressorDT=DecisionTreeClassifier(criterion='gini',min_impurity_split=1e-6,splitter='random')
Accuracy=[]
from sklearn.model_selection import KFold
from sklearn import metrics as mt
import statistics as st
kf=KFold(n_splits=40,shuffle=True)
for count in range(0,100):
  for train_index,val_index in kf.split(XTrain):
    XTrainKF,YTrainKF=XTrain[train_index],YTrain[train_index]
    XTestKF,YTestKF=XTrain[val_index],YTrain[val_index]
    RegressorDT.fit(XTrainKF,YTrainKF)
    YPredKF=RegressorDT.predict(XTestKF)
    Accuracy.append(mt.accuracy_score(YTestKF,YPredKF))
print("Average Accuracy: "+str(st.mean(Accuracy)))#0.8521388888888889

Average Accuracy: 0.8528055555555555


In [0]:
######### Predict ###############
RegressorDT.fit(XTrain,YTrain)
YPred=RegressorDT.predict(XTest)
cm=mt.confusion_matrix(YTest,YPred)
cm

/usr/local/lib/python3.6/dist-packages/sklearn/tree/tree.py:297: DeprecationWarning: The min_impurity_split parameter is deprecated. Its default value will change from 1e-7 to 0 in version 0.23, and it will be removed in 0.25. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)


array([[24,  3],
       [ 1, 12]])